# Explore weird pieces

Some pieces have weird contours that make it hard to find their corners. This space explores methods to split such contours into more manageable parts.


## Import


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loguru import logger as lg
from rich import get_console
from rich import print as rprint
from rich.console import Console

# some magic to make rich work in jupyter
# https://github.com/Textualize/rich/issues/3483
# enable it for every cell output with %load_ext rich
console: Console = get_console()
console.is_jupyter = False

In [ ]:
from pathlib import Path
import sys

import cv2
import matplotlib.pyplot as plt
import numpy as np

from snap_fit.aruco.aruco_board import ArucoBoardGenerator
from snap_fit.aruco.aruco_detector import ArucoDetector
from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.config.aruco.aruco_detector_config import ArucoDetectorConfig
from snap_fit.config.aruco.sheet_aruco_config import SheetArucoConfig
from snap_fit.config.types import EDGE_ENDS_TO_CORNER
from snap_fit.config.types import EdgePos
from snap_fit.image.process import find_contours
from snap_fit.image.process import find_corners
from snap_fit.image.segment_matcher import SegmentMatcher
from snap_fit.image.utils import draw_contour
from snap_fit.image.utils import draw_corners
from snap_fit.image.utils import show_image_mpl
from snap_fit.image.utils import show_images_mpl
from snap_fit.params.snap_fit_params import get_snap_fit_paths
from snap_fit.puzzle.piece import Piece
from snap_fit.puzzle.sheet import Sheet
from snap_fit.puzzle.sheet_aruco import SheetAruco

In [ ]:
snap_fit_paths = get_snap_fit_paths()

## Load Image Sheet


In [ ]:
# sheets_tag = "oca"
sheets_tag = "milano1"

In [ ]:
sheets_base_fol = snap_fit_paths.data_fol / sheets_tag
img_fol = sheets_base_fol / "sheets"

In [ ]:
# then in other notebooks reload it like this
sheet_aruco_config_fp = sheets_base_fol / f"{sheets_tag}_SheetArucoConfig.json"
lg.debug(f"Loading SheetArucoConfig from: {sheet_aruco_config_fp}")
sheet_config = SheetArucoConfig.model_validate_json(sheet_aruco_config_fp.read_text())
rprint(sheet_config)

In [ ]:
sheet_aruco = SheetAruco(sheet_config)

## Load some image


In [ ]:
img_idx = 1
img_fp = list(img_fol.iterdir())[img_idx]
lg.debug(f"Testing on image: {img_fp}")

In [ ]:
sheet = sheet_aruco.load_sheet(img_fp)

plt.figure(figsize=(5, 7))
plt.imshow(sheet.img_orig, cmap="gray")
plt.title("SheetAruco Loaded Image")
plt.axis("off")
plt.show()

In [ ]:
from snap_fit.image.utils import color_to_scalar

color = color_to_scalar(255, num_channels=3)
c_in = color_to_scalar(180, num_channels=3)

ps = sheet.pieces
imgs = []
for piece in [
    ps[i]
    for i in [
        0,
        # 1,
        # 2,
        # 3,
        # 4,
        5,
        6,
        # 7,
        # 8,
        # 9,
    ]
]:
    # for piece in [ps[i] for i in range(len(ps))]:
    # piece = sheet.pieces[0]
    # img_contour = draw_contour(piece.img_bw, piece.contour_loc, color=127)
    img_contour = draw_contour(piece.img_crossmasked, piece.contour_loc, color=127)
    corners = list(piece.corners.values())
    ic = draw_corners(img_contour, corners, color=190)

    p_box = [
        (0, 0),
        (ic.shape[1], 0),
        (ic.shape[1], ic.shape[0]),
        (0, ic.shape[0]),
    ]

    # for c in p_box:
    #     cv2.circle(ic, tuple(c), radius=20, color=color, thickness=3)

    # pad = 30
    # p_in = [
    #     (pad, pad),
    #     (ic.shape[1] - pad, pad),
    #     (pad, ic.shape[0] - pad),
    #     (ic.shape[1] - pad, ic.shape[0] - pad),
    # ]
    # for p in p_in:
    #     cv2.circle(ic, p, radius=99, color=c_in, thickness=4)

    # draw an ellipse using opencv
    p = {
        "startAngle": 0,
        "endAngle": 360,
        "color": c_in,
        "thickness": 1,
    }

    # ellipse_scale = 2.0

    for ellipse_scale in range(5, 150, 5):
        short_axis = int(1 * ellipse_scale)
        long_axis = int(1.4 * ellipse_scale)

        for i, pb in enumerate(p_box):
            cv2.ellipse(
                ic,
                center=pb,
                axes=(short_axis, long_axis),
                angle=-45 * (pow(-1, (i % 2))),
                **p,
            )

    imgs.append(ic)

show_images_mpl(imgs, figsize=(17, 17), columns=2)